In [ ]:
import math
import csv
import numpy as np # type: ignore
import pandas as pd # type: ignore
from sklearn.cluster import KMeans # type: ignore
from sklearn.metrics.pairwise import cosine_similarity # type: ignore
import time

ĐỌC VÀ XỬ LÝ DỮ LIỆU

In [2]:
data_ratings = pd.read_csv("data/ratings.dat", encoding="utf-8", sep='::', header=None, names=['userId','movieId','rating','tmp'], engine='python')
del data_ratings['tmp']

In [4]:
# Chuyển dữ liệu sang định dạng ma trận
ratings_matrix = data_ratings.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

# Tính ma trận tương tự cosin
similarity_matrix = cosine_similarity(ratings_matrix)

PHÂN CỤM DỮ LIỆU

In [5]:
# Phân cụm user sử dụng K-means
kmeans = KMeans(n_clusters=4, random_state=0).fit(similarity_matrix)

# Gán nhãn cho người dùng
user_clusters = pd.DataFrame({'userId': ratings_matrix.index, 'cluster': kmeans.labels_})

CHUẨN HOÁ DỮ LIỆU

In [6]:
# Chuyển đổi DataFrame sang từ điển
user_clusters_dict = user_clusters.set_index('userId').to_dict()['cluster']

In [ ]:
# Thêm cột cluster vào DataFrame
data_ratings['cluster'] = data_ratings['userId'].map(dict(zip(ratings_matrix.index, kmeans.labels_)))

In [8]:
# Chuyển đổi ma trận tương tự thành dataframe
similarity_df = pd.DataFrame(similarity_matrix, index=ratings_matrix.index, columns=ratings_matrix.index)

# Chuyển dataframe sang long format
similarity_df = similarity_df.reset_index().melt(id_vars='userId', value_name='similarity', var_name='userId2')

# Bỏ các hàng có userId bằng userId2
similarity_df = similarity_df[similarity_df['userId'] != similarity_df['userId2']]

# Đổi tên cột
similarity_df = similarity_df.rename(columns={'userId': 'u1', 'userId2': 'u2'})

# Sắp xếp các giá trị theo u1, u2
similarity_df = similarity_df.sort_values(['u1', 'u2']).reset_index(drop=True)


In [10]:
# Chuyển similarity_df sang dictionary để tăng tốc độ truy vấn
similarity_dict = similarity_df.set_index('u1').groupby(level=0).apply(lambda x: x.set_index('u2')['similarity'].to_dict()).to_dict()

GỢI Ý BỘ PHIM

In [11]:
def recommend_Movie_Nocluster(user):
    # Lấy danh sách các movieId
    movies = set(data_ratings['movieId'])
    rated_movies = set(data_ratings[data_ratings['userId'] == user]['movieId'])
    unrated_movies = movies - rated_movies # Chỉ lấy những phim chưa được đánh giá

    # Lọc danh sách độ tương đồng của các user khác với user cần dự đoán
    similarities = similarity_dict.get(user, {})
    
    # Merge dữ liệu để lấy các similarity scores
    merge_df = data_ratings[data_ratings['userId'].isin(similarities.keys())].copy()
    merge_df['similarity'] = merge_df['userId'].map(similarities)

    # Lọc những đánh giá của các bộ phim chưa được xem
    merge_df = merge_df[merge_df['movieId'].isin(unrated_movies)]

    # Tính tổng trọng số rating dựa trên độ tương đồng
    merge_df['weighted_rating'] = merge_df['rating'] * merge_df['similarity']

    recommendation_scores = merge_df.groupby('movieId').agg({'weighted_rating': 'sum'})

    # Lấy top 5 phim có tổng trọng số rating cao nhất
    top_movies = recommendation_scores['weighted_rating'].nlargest(5).index.tolist()

    if top_movies:
        return top_movies
    else:
        return None

In [13]:
def recommend_Movie_Cluster(user):

    # Lấy cluster của người dùng
    cluster = user_clusters_dict.get(user, {})

    # Lọc ra những người dùng khác trong cùng cluster
    cluster_users = data_ratings[(data_ratings['cluster'] == cluster)]

    # Lấy danh sách các movieId
    movies = set(cluster_users['movieId'])
    rated_movies = set(cluster_users[cluster_users['userId'] == user]['movieId'])
    unrated_movies = movies - rated_movies # Chỉ lấy những phim chưa được đánh giá
  
    # Lọc danh sách độ tương đồng của các user khác với user cần dự đoán
    similarities = similarity_dict.get(user, {})

    # Merge dữ liệu để lấy các user cùng cụm
    merge_df = cluster_users[cluster_users['userId'].isin(similarities.keys())].copy()
    merge_df['similarity'] = merge_df['userId'].map(similarities)

    # Lọc những đánh giá của các bộ phim chưa được xem
    merge_df = merge_df[merge_df['movieId'].isin(unrated_movies)]

    # Tính tổng trọng số rating dựa trên độ tương đồng
    merge_df['weighted_rating'] = merge_df['rating'] * merge_df['similarity']

    recommendation_scores = merge_df.groupby('movieId').agg({'weighted_rating': 'sum'})

    # Lấy top 5 phim có tổng trọng số rating cao nhất
    top_movies = recommendation_scores['weighted_rating'].nlargest(5).index.tolist()

    if top_movies:
        return top_movies
    else:
        return None

THỜI GIAN GỢI Ý BỘ PHIM

In [14]:
# Loại bỏ các dòng có userId trùng lặp
unique_data = data_ratings.drop_duplicates(subset=['userId'])

In [15]:
def calculate_Time_Recommend_Nocluster():
    recommend = []
    for index, row in unique_data.iterrows():
        user_id = row['userId']
        start_time = time.time()
        recommend_movies = recommend_Movie_Nocluster(user_id)
        end_time = time.time()
        prediction_time = end_time - start_time
        recommend.append({'prediction_time': prediction_time})
        
    recommed_df = pd.DataFrame(recommend)
    mean_recommend_time = recommed_df['prediction_time'].mean()
    return  mean_recommend_time

# Tính toán dự đoán rating cho tất cả các cặp user-movie
mean_recommend_time = calculate_Time_Recommend_Nocluster()
print("mean_prediction_time: ",mean_recommend_time)

mean_prediction_time:  0.1572233236388655


In [16]:
def calculate_Time_Recommend_Cluster():
    recommend = []
    for index, row in unique_data.iterrows():
        user_id = row['userId']
        start_time = time.time()
        recommend_movies = recommend_Movie_Cluster(user_id)
        end_time = time.time()
        prediction_time = end_time - start_time
        recommend.append({'prediction_time': prediction_time})
        
    recommed_df = pd.DataFrame(recommend)
    mean_recommend_time = recommed_df['prediction_time'].mean()
    return  mean_recommend_time

# Tính toán dự đoán rating cho tất cả các cặp user-movie
mean_recommend_time = calculate_Time_Recommend_Cluster()
print("mean_prediction_time: ",mean_recommend_time)

mean_prediction_time:  0.0467858336619194


ĐÁNH GIÁ ĐỘ LỆCH GỢI Ý PHIM

In [15]:
def calculate_total_weighted_recommendation(user, list_movies):
    # Lấy danh sách các movieId
    movies = set(data_ratings['movieId'])
    rated_movies = set(data_ratings[data_ratings['userId'] == user]['movieId'])
    unrated_movies = movies - rated_movies  # Chỉ lấy những phim chưa được đánh giá

    # Lọc danh sách độ tương đồng của các user khác với user cần dự đoán
    similarities = similarity_dict.get(user, {})

    # Merge dữ liệu để lấy các similarity scores
    merge_df = data_ratings[data_ratings['userId'].isin(similarities.keys())].copy()
    merge_df['similarity'] = merge_df['userId'].map(similarities)

    # Lọc những đánh giá của các bộ phim chưa được xem
    merge_df = merge_df[merge_df['movieId'].isin(unrated_movies)]

    # Tính tổng trọng số rating dựa trên độ tương đồng
    merge_df['weighted_rating'] = merge_df['rating'] * merge_df['similarity']

    # Tính tổng trọng số rating cho từng movieId
    recommendation_scores = merge_df.groupby('movieId').agg({'weighted_rating': 'sum'})

    # Tổng trọng số của tất cả các movieId
    total_weighted_rating_movie = recommendation_scores['weighted_rating'].sum()
    
    # Lọc recommendation_scores dựa vào danh sách movie_ids
    filtered_scores = recommendation_scores.loc[list_movies]

    # Tính tổng trọng số rating cho danh sách movie_ids
    total_weighted_rating = filtered_scores['weighted_rating'].sum()

    return total_weighted_rating, total_weighted_rating_movie

TỈ LỆ CHÊNH LỆCH CHO TỪNG USER

In [ ]:
def generate_comparison_dataframe_user(user_list):
    # Khởi tạo danh sách để lưu kết quả
    results = []
    # Lặp qua từng user trong danh sách user_list
    for user in user_list:
        # Tính tổng trọng số từ phương pháp không phân cụm
        total_weighted_nocluster, total_weighted_rating_movie = calculate_total_weighted_recommendation(user, recommend_Movie_Nocluster(user))
        
        # Tính tổng trọng số từ phương pháp có phân cụm
        total_weighted_cluster, total_weighted_rating_movie= calculate_total_weighted_recommendation(user, recommend_Movie_Cluster(user))

        # Tính tổng trọng số lệch (độ chênh lệch tuyệt đối)
        total_weighted_difference = abs(total_weighted_nocluster - total_weighted_cluster)

        # Tính tỉ lệ chênh lệch giữa hai phương pháp
        if total_weighted_rating_movie != 0:
            ratio_difference = (total_weighted_difference* 100) / total_weighted_rating_movie
        else:
            ratio_difference = 0  # Xử lý khi tổng bằng 0 để tránh lỗi chia cho 0

        # Thêm kết quả cho từng user vào danh sách
        results.append({
            "User": user,
            "Tổng trọng số không phân cụm": f"{total_weighted_nocluster:.6f}",
            "Tổng trọng số có phân cụm": f"{total_weighted_cluster:.6f}",
            "Tổng trọng số tất cả movie": f"{total_weighted_rating_movie:.6f}",
            "Tỉ lệ chênh lệch (%)": f"{ratio_difference:.2f}%"
        })
    # Chuyển danh sách kết quả thành DataFrame
    df = pd.DataFrame(results)
    df.to_excel('ratio_movie.xlsx')
    return df

TỈ LỆ CHÊNH LỆCH CỦA TẤT CẢ CÁC USER

In [ ]:
all_users = data_ratings['userId'].unique()

In [ ]:
def generate_comparison_dataframe_all_user(user_list):
    # Khởi tạo danh sách để lưu kết quả
    results = []
    count_total_nocluster = 0
    count_total_cluster = 0
    count_total_weighted_rating_movie = 0
    # Lặp qua từng user trong danh sách user_list
    for user in user_list:
        # Tính tổng trọng số từ phương pháp không phân cụm
        total_weighted_nocluster, total_weighted_rating_movie = calculate_total_weighted_recommendation(user, recommend_Movie_Nocluster(user))
        count_total_nocluster += total_weighted_nocluster
        # Tính tổng trọng số từ phương pháp có phân cụm
        total_weighted_cluster, total_weighted_rating_movie = calculate_total_weighted_recommendation(user, recommend_Movie_Cluster(user))
        count_total_cluster += total_weighted_cluster
        # Tính tổng trọng số của cả hai phương pháp
        count_total_weighted_rating_movie += total_weighted_rating_movie
    # Tính tổng trọng số lệch (độ chênh lệch tuyệt đối)
    total_weighted_difference = abs(count_total_nocluster - count_total_cluster)

    # Tính tỉ lệ chênh lệch giữa hai phương pháp
    if total_weighted_rating_movie != 0:
        ratio_difference = (total_weighted_difference*100) / count_total_weighted_rating_movie
    else:
        ratio_difference = 0  # Xử lý khi tổng bằng 0 để tránh lỗi chia cho 0
    
    # Thêm kết quả cho từng user vào danh sách
    results.append({
        "Tổng User": len(user_list),
        "Tổng trọng số không phân cụm của tất cả user": f"{count_total_nocluster:,.0f}",
        "Tổng trọng số có phân cụm của tất cả user": f"{count_total_cluster:,.0f}",
        "Tổng trọng số tất cả movie của tất cả user": f"{count_total_weighted_rating_movie:,.0f}",
        "Tỉ lệ chênh lệch (%)": f"{ratio_difference:.2f}%"
    })
    # Chuyển danh sách kết quả thành DataFrame
    df = pd.DataFrame(results)
    return df